## Naomi, Parshana, Sinchana, Joanna

In [ ]:
%pip install pandas numpy matplotlib scikit-learn keras tensorflow nltk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

# Download NLTK resources if not already downloaded
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


## TASK 1 - EDA

### Question 1

In [64]:
df_main = pd.read_csv("S19_All_Release_2_10_22/Data/MainTable.csv")
df_subj = pd.read_csv("S19_All_Release_2_10_22/Data/LinkTables/Subject.csv")
df_problems = pd.read_csv("2nd CSEDM Data Challenge - Problem Prompts & Concepts Used - Sheet1.CSV")

In [ ]:
# Number of unique students - using MainTable.csv
len(df_main['SubjectID'].unique())

In [ ]:
# Number of unique students - using LinkTables/Subject.csv
len(df_subj['SubjectID'].unique())
# Less because some people did not take the final

### Question 2

In [67]:
df_early = pd.read_csv("S19_All_Release_2_10_22/early.csv")
df_late = pd.read_csv("S19_All_Release_2_10_22/late.csv")
df_all = pd.concat([df_early, df_late])

In [ ]:
mean_attempts = df_all.groupby(['AssignmentID', 'ProblemID'])['Attempts'].mean()
mean_attempts.head()
print(f"Assignment {mean_attempts.idxmax()[0]} Problem {mean_attempts.idxmax()[1]} had the most attempts at {mean_attempts.max()}.")

### Question 3

In [69]:
df_main = pd.read_csv("S19_All_Release_2_10_22/Data/MainTable.csv")
df_compile = df_main[df_main['EventType'] == "Compile"]

In [ ]:
compile_errors = df_compile[df_compile['Compile.Result'] == 'Error']
grouped_compile_errors = compile_errors.groupby(['AssignmentID', 'ProblemID'])['Compile.Result'].size()
print(f"Assignment {grouped_compile_errors.idxmax()[0]} Problem {grouped_compile_errors.idxmax()[1]} had the most compile errors at {grouped_compile_errors.max()}.")

Computing the total number of compiler errors faced on each problem, and report the maximum is more meaningful because it directly answers which problems had the most compile errors from all students. This is more meaningful for knowing which problems students as a whole struggled with compiling.

## Task 2 - Open-Ended Analysis

### Research Analysis 1: Analyzing frequency of errors changing over time

In [71]:
messages = df_main[df_main['EventType'] == "Compile.Error"][['ServerTimestamp','AssignmentID', 'ProblemID', 'CompileMessageData']]

messages['ServerTimestamp'] = pd.to_datetime(messages['ServerTimestamp'])
messages['Date'] = messages['ServerTimestamp'].dt.date

In [ ]:
messages['CompileMessageData'] = messages['CompileMessageData'].apply(
    lambda x: re.sub(r"line \d+: (error: |\[empty\])?", "", x)
)
messages

CCategorize compile error messages into different types:

In [73]:
def categorize_error(msg):
    # Syntax Error
    if re.search(r"(illegal|cannot find symbol|character literal|illegal start of|illegal escape character|illegal character|unclosed string literal|illegal line end in character literal|invalid method declaration; return type required\.|array dimension missing|bad operand type int for unary operator '!'|bad initializer for for-loop|array required, but int found|no suitable constructor found for String\(char\)|not a statement|missing return statement|integer number too large)", msg, re.IGNORECASE):
        return "Syntax Error"
    # Type Error
    elif re.search(r"(bad operand type|dereferenced|type not allowed|unexpected type|char cannot be dereferenced|incompatible types|incomparable types|cannot be converted|bad operand types|int cannot be dereferenced|array dimension missing|incomparable types: boolean and int)", msg, re.IGNORECASE):
        return "Type Error"
    # Variable/Method not found
    elif re.search(r"(variable .+ is already defined|cannot assign a value to final variable|cannot assign a value to final variable this|cannot find symbol: method|cannot find symbol: variable|cannot find symbol: class)", msg, re.IGNORECASE):
        return "Variable/Method not found"
    # Initialization Issues
    elif re.search(r"(variable declaration not allowed here|variable .+ might not have been initialized|invalid method declaration|class expected|class .+ is already defined)", msg, re.IGNORECASE):
        return "Initialization Issues"
    # Control Flow
    elif re.search(r"(for-each not applicable to expression type|orphaned case|break outside switch or loop|unreachable statement|'else' without 'if'|reached end of file|empty statement after if|'finally' without 'try'|return outside method|while expected)", msg, re.IGNORECASE):
        return "Control Flow"
    # Class/Interface declaration
    elif re.search(r"(class, interface, or enum expected)", msg, re.IGNORECASE):
        return "Class/Interface declaration"
    # Method Argument Errors
    elif re.search(r"(required, but|missing method|method .+ already defined|method .+ cannot be applied to given types|no arguments|no suitable method found)", msg, re.IGNORECASE):
        return("Method Argument")
    # Character expected
    elif re.search(r"(: expected|';' expected|'\)' expected|'\]' expected|'\[' expected|'\(' expected|; expected|> expected|' expected)", msg, re.IGNORECASE):
        return("Char Expected")
    else:
        return "Other"
messages['ErrorCategory'] = messages['CompileMessageData'].apply(categorize_error)

In [ ]:
error_counts = messages.groupby(['Date', 'ErrorCategory']).size().unstack(fill_value=0)

# Plotting the line graph
plt.figure(figsize=(10, 6))
for category in error_counts.columns:
    plt.plot(error_counts.index, error_counts[category], label=category)

plt.xlabel('Date')
plt.ylabel('Number of Errors')
plt.title('Error Categories Over Time')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.legend(title='Error Category')
plt.tight_layout()
plt.show()

We see spikes in errors presumably when an assignment is due, and students are compiling their code more often before the due date. The syntax errors and char expected errors are significantly higher than other types of errors. This is expected since novice programmers likely haven’t written enough code to check syntax thoroughly before compiling. Even advanced programmers forget a semicolon every now and then. Char expected errors also go hand in hand with syntax errors and can be linked to the same cause. 

In [ ]:
exclude_messages = messages[~messages['ErrorCategory'].isin(['Syntax Error', 'Char Expected'])]
error_counts_excluded = exclude_messages.groupby(['Date', 'ErrorCategory']).size().unstack(fill_value=0)

plt.figure(figsize=(10, 6))
for category in error_counts_excluded.columns:
    plt.plot(error_counts_excluded.index, error_counts_excluded[category], label=category)

plt.xlabel('Date')
plt.ylabel('Number of Errors')
plt.title('Error Categories Over Time - Excluding Syntax Error and Char Expected')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.legend(title='Error Category')
plt.tight_layout()
plt.show()

This graph is the same as the one above, but without syntax and char expected errors, which were an overwhelming majority, clouding our analysis. We now see a new trend in control flow and type errors. However, only type errors spike halfway through the term in this graph. This can be compared with lecture material and lab assignments to find out which assignment in particular cause confusion among students. We can hypothesize that there might have been new types introduced that the students struggled with. All of the other types of errors are relatively low but they do not necessarily decrease as the term goes on. In fact, they get higher midway through, then drop low again. This leads us to believe that there was either a very challenging assignment halfway through the quarter, or some new concept was introduced that really altered the students’ understanding of concepts they excelled at before. 


### Research Analysis 2: Predicting Score from problem features

In [ ]:
df_subj

In [ ]:
df_main

In [ ]:
df_score = df_main[df_main['EventType'] == "Run.Program"]
df_score = df_score.groupby(['SubjectID', 'AssignmentID', 'ProblemID'])['Score'].max().reset_index()
df_score.head()

In [ ]:
score_problems = pd.merge(df_score, df_problems, on=['AssignmentID', 'ProblemID'], how='inner')
score_problems.head()

In [ ]:
df_all

In [ ]:
df_problems.head()

In [ ]:
## collapse to one row per student by averaging the number of attempts per problem

df_grades = pd.merge(df_score, df_all, on=['SubjectID', 'AssignmentID', 'ProblemID'], how='inner')
#df_grades = df_grades.groupby(['SubjectID', 'AssignmentID', 'ProblemID'])[['Attempts', 'Score']].max()
df_grades = pd.merge(df_grades, df_subj, on=['SubjectID'], how='inner')
df_grades = pd.merge(df_grades, df_problems, on=['AssignmentID', 'ProblemID'], how='inner')
df_grades.columns

In [83]:
important_columns = ['SubjectID', 'AssignmentID', 'ProblemID', 'Attempts', 'Score', 'X-Grade','Requirement', 'If/Else',
       'NestedIf', 'While', 'For', 'NestedFor', 'Math+-*/', 'Math%',
       'LogicAndNotOr', 'LogicCompareNum', 'LogicBoolean', 'StringFormat',
       'StringConcat', 'StringIndex', 'StringLen', 'StringEqual', 'CharEqual',
       'ArrayIndex', 'DefFunction']

In [ ]:
df_model = df_grades[important_columns]
df_model.head()
df_max = df_model.loc[df_model.groupby(['SubjectID', 'AssignmentID', 'ProblemID'])['Score'].idxmax()]
df_max.head()

In [ ]:
df_max.columns

In [ ]:
def replace_problem_details_with_score(df):
    problem_detail_cols = ['If/Else', 'NestedIf', 'While', 'For',
       'NestedFor', 'Math+-*/', 'Math%', 'LogicAndNotOr', 'LogicCompareNum',
       'LogicBoolean', 'StringFormat', 'StringConcat', 'StringIndex',
       'StringLen', 'StringEqual', 'CharEqual', 'ArrayIndex', 'DefFunction']

    for col in problem_detail_cols:
        df[col] = df.apply(lambda row: row['Score'] if row[col] == 1 else row[col], axis=1)
    
    return df

df_updated = replace_problem_details_with_score(df_max)

df_updated.head()

In [87]:
df_model = df_updated.groupby(['SubjectID', 'X-Grade'])[['If/Else',
       'NestedIf', 'While', 'For', 'NestedFor', 'Math+-*/', 'Math%',
       'LogicAndNotOr', 'LogicCompareNum', 'LogicBoolean', 'StringFormat',
       'StringConcat', 'StringIndex', 'StringLen', 'StringEqual', 'CharEqual',
       'ArrayIndex', 'DefFunction']].mean()

In [ ]:
df_model.head()

### Research Analysis 2 Part 2: Start building the model (s)

In [ ]:
df_model = df_model.fillna(df_model.median())  # or df.fillna(df.median())

df_model.reset_index(inplace=True)

y = df_model['X-Grade']

X = df_model[['If/Else',
       'NestedIf', 'While', 'For', 'NestedFor', 'Math+-*/', 'Math%',
       'LogicAndNotOr', 'LogicCompareNum', 'LogicBoolean', 'StringFormat',
       'StringConcat', 'StringIndex', 'StringLen', 'StringEqual', 'CharEqual',
       'ArrayIndex', 'DefFunction']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual vs Predicted Grades - Linear Regression')
#plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.show()
RMSE = root_mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {RMSE}')



In [ ]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_knn)
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual vs Predicted Grades (KNN)')

RMSE = root_mean_squared_error(y_test, y_pred_knn)
print(f'Root Mean Squared Error: {RMSE}')


In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_rf)
plt.title('Actual vs Predicted Grades (Random Forest)')
RMSE = root_mean_squared_error(y_test, y_pred_rf)
print(f'Root Mean Squared Error: {RMSE}')


In [ ]:
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_gb)
plt.title('Actual vs Predicted Grades (Gradient Boosting)')
RMSE = root_mean_squared_error(y_test, y_pred_gb)
print(f'Root Mean Squared Error: {RMSE}')

In [ ]:
# plot residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_gb, y_test - y_pred_gb)
plt.xlabel('Predicted Grades')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Grades (Gradient Boosting)')
plt.axhline(y=0, color='r', linestyle='--')


In [ ]:
nn = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
nn.fit(X_train, y_train)
y_pred_nn = nn.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_nn)
plt.title('Actual vs Predicted Grades (Neural Network)')
RMSE = root_mean_squared_error(y_test, y_pred_nn)
print(f'Root Mean Squared Error: {RMSE}')

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['root_mean_squared_error'])

history = model.fit(X_train, y_train, epochs=250, batch_size=32, validation_split=0.2, verbose=1)

y_pred_nn = model.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_nn)
plt.title('Actual vs Predicted Grades (Neural Network)')
RMSE = root_mean_squared_error(y_test, y_pred_nn)
print(f'Root Mean Squared Error: {RMSE}')

In [ ]:
# plot residuals
y_pred_nn = y_pred_nn.flatten()
plt.figure(figsize=(10, 6))
residuals = y_test - y_pred_nn
plt.scatter(y_pred_nn, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Grades')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Grades')
plt.show()


In [ ]:
# plot train vs test RMSE
plt.plot(history.history['root_mean_squared_error'][200:], label='train')
plt.plot(history.history['val_root_mean_squared_error'][200:], label='validation')

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Train vs Validation RMSE')
plt.show()

### Research Analysis 2 Part 3: Attempting TFIDF

In [ ]:
df_max.head()

In [99]:
## find average score and attempts per problem
df_problem_avg = df_max.groupby(['AssignmentID','ProblemID', 'Requirement'])[['Score', 'Attempts']].mean().reset_index()

In [100]:
## problem wording preprocessing
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def custom_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemma.lemmatize(w.lower()) for w in tokens if w not in stop_words]
    return tokens

In [ ]:
#X = vectorizer.fit_transform(df_problem_avg['Requirement'])
#df_problem_avg['tokenized'] = vectorizer.fit_transform(df_problem_avg['Requirement']).toarray().tolist()

vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
mms = MinMaxScaler()
X_text = vectorizer.fit_transform(df_problem_avg['Requirement']).toarray()  # Convert to NumPy array
med = df_problem_avg['Score'].median()
df_problem_avg['score_cat'] = df_problem_avg['Score'].apply(lambda x: 1 if x > med else 0)

# Add additional numerical features
#X_attempts = df_problem_avg[['Attempts']].values
X_attempts = mms.fit_transform(df_problem_avg[['Attempts']])

X = np.hstack((X_text, X_attempts))

y_cat = df_problem_avg['score_cat']
y = df_problem_avg['Score']
#X = df_problem_avg[['tokenized', 'Attempts']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X, y_cat, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual vs Predicted Score - Linear Regression')
plt.show()
RMSE = root_mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {RMSE}')

In [ ]:
# plot residuals

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, y_test - y_pred)
plt.xlabel('Predicted Grades')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Grades')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
feature_names = vectorizer.get_feature_names_out()

# Get model coefficients
coefficients = model.coef_[:-1]  # Exclude the last coefficient (Attempts)

# Create a DataFrame for analysis
feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort by most negative coefficients (words that lead to worse scores)
feature_importance = feature_importance.sort_values(by='Coefficient')

# Display the top negative words
top_negative_words = feature_importance.head(20)

# Plot the results
plt.figure(figsize=(12, 6))
plt.barh(top_negative_words['Feature'], top_negative_words['Coefficient'], color='red')
plt.xlabel('Coefficient Value')
plt.ylabel('Words')
plt.title('Top Words Associated with Lower Scores')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_problem_avg[df_problem_avg['Requirement'].str.contains('word', case=False, na=False)]#['Requirement'].tolist()

In [ ]:
df_problem_avg[df_problem_avg['Requirement'].str.contains('word', case=False, na=False)]['Requirement'].tolist()[0]

**This problem has the worst performance**

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_knn)
plt.xlabel('Actual Grades')
plt.ylabel('Predicted Grades')
plt.title('Actual vs Predicted Grades (KNN)')
RMSE = root_mean_squared_error(y_test, y_pred_knn)
print(f'Root Mean Squared Error: {RMSE}')

### Research Analysis 3: Investigating questions that had the highest number of variance and theorizing why students might have a high level of variance

In [ ]:
df_early = pd.read_csv("S19_All_Release_2_10_22/early.csv")
df_late = pd.read_csv("S19_All_Release_2_10_22/late.csv")
df_subject = pd.read_csv("S19_All_Release_2_10_22/Data/LinkTables/Subject.csv")
df_all = pd.concat([df_early, df_late])
df = pd.merge(df_all, df_subject, left_on = 'SubjectID', right_on= 'SubjectID')
df

In [ ]:
df = df.sort_values(by="AssignmentID")
attempt_variance = df.groupby(['AssignmentID', 'ProblemID'])['Attempts'].var().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='ProblemID', y='Attempts', hue='AssignmentID', data=attempt_variance, palette='Set2', order=attempt_variance.sort_values(['AssignmentID', 'ProblemID'])['ProblemID'])
plt.xlabel("Problem ID")
plt.ylabel("Variance in Attempts")
plt.title("Variance in Attempts Per Problem Across Assignments")
plt.xticks(rotation=90)
plt.legend(title="Assignment ID")
plt.show()
# Investigate problems 101, 102 as the varrying number of students from high to low amount of attempts to significantly bigger than others

This graph shows the variance in attempts for each question. Questions 101 and 102 had the highest variance by far, which means some students got it immediately, while other took several tries before they got the question correct. We can infer that these questions may have been about a concept that some students were not understanding clearly, or that there was an issue in the way the questions were worded. There was clearly a misunderstanding on the educator’s part either about the students’ understanding of the concept, or the clarity of the assignment. 

In [ ]:
df_101 = df[(df['ProblemID'] == 101) & (df['AssignmentID'] == 487)]
sns.boxplot(x=df_101['Attempts'], showfliers=False)

Problem 101 Question: “We want to make a package of goal kilos of chocolate. We have small bars (1 kilo each) and big bars (5 kilos each). Return the number of small bars to use, assuming we always use big bars before small bars. Return -1 if it can't be done.”

This is a box plot of the attempts on question 101. It shows that 50% of students got between 1-6 attempts, 50% of students got between 6-20 attempts. These are wide quartiles and prompt us to look into what caused so much confusion. After reading the question, we can see that it was difficult to understand what was being asked of the students. The first sentence doesn’t give enough context for students to know what they are looking for, or returning, in the code. This is the first point of confusion. The questions continues on to explain what types of bars there are and the rules on how many of each to use. Students likely made assumptions about how many big bars to use, and what the big and small bars must add up to. After this analysis, it is more than likely that the question itself was worded in a way that made it difficult for students to understand. We can infer that most students got it correct after understanding what to do, but that is the part that caused so much variance in number of attempts. In the future, the educator should either clarify the question or not use it anymore. 

In [ ]:
df_102 = df[(df['ProblemID'] == 102) & (df['AssignmentID'] == 487)]
sns.boxplot(x=df_102['Attempts'], showfliers=False)

Problem 102 Question: Given a string, if the same 2-character substring appears at both its beginning and end, return the contents of the string without the 2-character substring at the beginning. For example, "HelloHe" yields "lloHe". Note that the 2-character substring at the beginning may overlap with the one at the end, so "Hi" yields "". If the two characters at the start and end of the string do not match each other, return the original string unchanged.

Question 102 asks for string manipulation. 50% of students got between 1-7 attempts, 50% of students got between 8-38 attempts. While the question itself is relatively clear, it does seem like there would be some edge cases, one of which is clarified in the prompt. For example, case sensitivity is not clarified in the question, so some students may have changed all the letters to lowercase for matching purposes. Additionally, strings such at “aa” or “” will return empty strings which could also cause issues for novice programmers. Test cases could also include cases with strings of length 1, which would return the same string. However, if the program the student wrote does not check string length then it would throw an error. This question has some ambiguity on how certain cases are handled which we can assume is the cause for large variance in attempts. 
